In [125]:
import pandas as pd
import math
def y_func(unagg: pd.DataFrame, page_cols: list, group_cols: list, s: list, sum_cols: list):
    assert s != []
    current_group_cols = group_cols + [page_cols[ss-1] for ss in s]
    if group_cols == []:
        y = (unagg.groupby(current_group_cols).agg({sum_col: "sum" for sum_col in sum_cols}) ** 2).agg({sum_col: "sum" for sum_col in sum_cols})
    else:
        y = (unagg.groupby(current_group_cols).agg({sum_col: "sum" for sum_col in sum_cols}) ** 2).groupby(group_cols).agg({sum_col: "sum" for sum_col in sum_cols})
    return y

def b_func(p: list, s: list):
    out = 1
    for i in range(1, len(p)+1):
        if i in s:
            out *= p[i-1]
        else:
            out *= p[i-1]**2
    return out

def c_func(p: list, s: list):
    # iterate over every subset of s
    out = (-1)**len(s) * b_func(p, [])
    for i in range(1, 2**len(s)):
        subset = [s[j] for j in range(len(s)) if i & (1 << j)]
        out += (-1)**(len(subset) + len(s)) * b_func(p, subset)
    return out

def compute_y(unagg: pd.DataFrame, page_cols: list, group_cols: list, sum_cols: list):
    s = [i for i in range(1, len(page_cols)+1)]
    # compute y on every subset of s and store it
    y = {}
    for i in range(1, 2**len(s)):
        subset = [s[j] for j in range(len(s)) if i & (1 << j)]
        y[tuple(subset)] = y_func(unagg, page_cols, group_cols, list(subset), sum_cols)
    return y

def var_func(p: list, y: dict):
    out = 0
    pp = math.prod(p) ** 2
    s = [i for i in range(1, len(p)+1)]
    for i in range(1, 2**len(s)):
        subset = [s[j] for j in range(len(s)) if i & (1 << j)]
        out += c_func(p, subset) * y[tuple(subset)] / pp
    return out

# Query 12

In [118]:
import pandas as pd
q12_unagg = pd.read_csv('/mydata/gt_query_12.csv')
q12_unagg

,l_shipmode,sum_1,sum_2,l_pageid,o_pageid
0,MAIL,0,1,17,4
1,MAIL,1,0,20,4
2,MAIL,1,0,21,5
3,MAIL,0,1,24,5
4,MAIL,1,0,42,9
...,...,...,...,...,...
28637880,SHIP,1,0,115517720,26843239
28637881,SHIP,0,1,115517726,26843241
28637882,SHIP,1,0,115517730,26843241
28637883,SHIP,0,1,115517736,26843243


In [119]:
y=compute_y(q12_unagg, ["l_pageid", "o_pageid"], ["l_shipmode"], ["sum_1", "sum_2"])
y

{(1,):               sum_1     sum_2
 l_shipmode                   
 MAIL        6923104  10606590
 SHIP        6923838  10608204,
 (2,):               sum_1     sum_2
 l_shipmode                   
 MAIL        8059112  13139386
 SHIP        8057946  13145566,
 (1,
  2):               sum_1     sum_2
 l_shipmode                   
 MAIL        6896718  10547420
 SHIP        6897346  10549530}

In [92]:
from scipy.stats import norm
z_val = norm.ppf(0.975)
max_var = (0.05 * q12_unagg.groupby("l_shipmode").agg({"sum_1": "sum", "sum_2": "sum"}) /  z_val) ** 2

In [120]:
search_space = [0.0001 * i for i in range(1,10)] + [0.001 * i for i in range(1,10)] + [0.01 * i for i in range(1,10)] + [0.1 * i for i in range(1,10)]
# sample lineitem only
for p1 in search_space:
    current_var = var_func([p1, 1], y)
    if (current_var <= max_var).all().all():
        lineitem_only = (p1, 1)
        break
print(lineitem_only)

(0.00030000000000000003, 1)


In [121]:
# sample orders only
for p2 in search_space:
    current_var = var_func([1, p2], y)
    if (current_var <= max_var).all().all():
        orders_only = (1, p2)
        break
print(orders_only)

(1, 0.0004)


In [122]:
lineitem_orders = []
for p1 in search_space:
    for p2 in search_space:
        current_var = var_func([p1, p2], y)
        if (current_var <= max_var).all().all():
            lineitem_orders.append((p1, p2))
            break
print(lineitem_orders)

[(0.0004, 0.7000000000000001), (0.0005, 0.6000000000000001), (0.0006000000000000001, 0.5), (0.0007, 0.4), (0.0008, 0.4), (0.0009000000000000001, 0.4), (0.001, 0.30000000000000004), (0.002, 0.2), (0.003, 0.1), (0.004, 0.07), (0.005, 0.06), (0.006, 0.05), (0.007, 0.04), (0.008, 0.04), (0.009000000000000001, 0.04), (0.01, 0.03), (0.02, 0.02), (0.03, 0.01), (0.04, 0.007), (0.05, 0.006), (0.06, 0.005), (0.07, 0.004), (0.08, 0.004), (0.09, 0.004), (0.1, 0.003), (0.2, 0.002), (0.30000000000000004, 0.001), (0.4, 0.0008), (0.5, 0.0006000000000000001), (0.6000000000000001, 0.0006000000000000001), (0.7000000000000001, 0.0005), (0.8, 0.0004), (0.9, 0.0004)]


In [123]:
possible_sample_rates = [lineitem_only, orders_only] + lineitem_orders
with open("q12_sample_rates.csv", "w") as f:
    f.write("p1,p2\n")
    for rate in possible_sample_rates:
        f.write(f"{rate[0]*100:.2f},{rate[1]*100:.2f}\n")

# Query 14

In [20]:
import pandas as pd
q14_unagg = pd.read_csv('/mydata/gt_query_14.csv')
q14_unagg

,sum_1,sum_2,l_pageid,p_pageid
0,0.00,3503.1864,3,3876625
1,0.00,61838.1720,4,2892462
2,0.00,36125.7600,4,2991546
3,0.00,36869.0896,6,398927
4,0.00,47491.9008,6,707431
...,...,...,...,...
74818883,0.00,42131.5426,115517739,3647695
74818884,0.00,20705.0580,115517742,542980
74818885,0.00,1085.9580,115517744,2299369
74818886,2319896.84,23198.9684,115517746,914345


In [127]:
y = compute_y(q14_unagg, ["l_pageid", "p_pageid"], [], ["sum_1", "sum_2"])

In [23]:
# lineitem: 1
# orders: 2
y1 = (q14_unagg.groupby(["l_pageid"]).agg({"sum_1": "sum", "sum_2": "sum"}) ** 2).agg({"sum_1": "sum", "sum_2": "sum"})
y2 = (q14_unagg.groupby(["p_pageid"]).agg({"sum_1": "sum", "sum_2": "sum"}) ** 2).agg({"sum_1": "sum", "sum_2": "sum"})
y12 = (q14_unagg.groupby(["l_pageid", "p_pageid"]).sum() ** 2).agg({"sum_1": "sum", "sum_2": "sum"})
y1, y2, y12

(sum_1    2.660162e+20
 sum_2    2.795639e+17
 dtype: float64,
 sum_1    7.991098e+20
 sum_2    2.003036e+18
 dtype: float64,
 sum_1    2.260445e+20
 sum_2    1.355492e+17
 dtype: float64)

In [128]:
from scipy.stats import norm
z_val = norm.ppf(0.975)
# var = lambda p1, p2: (p2 - p1*p2) / (p1*p2) * y1 + (p1 - p1*p2) / (p1*p2) * y2 + (1 - p1 - p2 + p1*p2) / (p1*p2) * y12
max_var = (0.05 * q14_unagg.agg({"sum_1": "sum", "sum_2": "sum"}) /  z_val) ** 2
print(max_var)

sum_1    1.336908e+24
sum_2    4.808507e+21
dtype: float64


In [129]:
search_space = [0.0001 * i for i in range(1,10)] + [0.001 * i for i in range(1,10)] + [0.01 * i for i in range(1,10)] + [0.1 * i for i in range(1,10)]
# sample lineitem only
for p1 in search_space:
    current_var = var_func([p1, 1], y)
    if (current_var <= max_var).all().all():
        lineitem_only = (p1, 1)
        break
print(lineitem_only)

(0.0002, 1)


In [130]:
# sample orders only
for p2 in search_space:
    current_var = var_func([1, p2], y)
    if (current_var <= max_var).all().all():
        part_only = (1, p2)
        break
print(part_only)

(1, 0.0006000000000000001)


In [131]:
lineitem_part = []
for p1 in search_space:
    for p2 in search_space:
        current_var = var_func([p1, p2], y)
        if (current_var <= max_var).all().all():
            lineitem_part.append((p1, p2))
            break
print(lineitem_part)

[(0.00030000000000000003, 0.7000000000000001), (0.0004, 0.5), (0.0005, 0.4), (0.0006000000000000001, 0.30000000000000004), (0.0007, 0.30000000000000004), (0.0008, 0.30000000000000004), (0.0009000000000000001, 0.2), (0.001, 0.2), (0.002, 0.09), (0.003, 0.06), (0.004, 0.05), (0.005, 0.04), (0.006, 0.03), (0.007, 0.03), (0.008, 0.03), (0.009000000000000001, 0.02), (0.01, 0.02), (0.02, 0.009000000000000001), (0.03, 0.007), (0.04, 0.005), (0.05, 0.004), (0.06, 0.004), (0.07, 0.003), (0.08, 0.003), (0.09, 0.003), (0.1, 0.003), (0.2, 0.002), (0.30000000000000004, 0.001), (0.4, 0.0009000000000000001), (0.5, 0.0008), (0.6000000000000001, 0.0008), (0.7000000000000001, 0.0007), (0.8, 0.0007), (0.9, 0.0007)]


In [135]:
possible_sample_rates = [lineitem_only, part_only] + lineitem_part
with open("q14_sample_rates.csv", "w") as f:
    f.write("p1,p2\n")
    for rate in possible_sample_rates:
        f.write(f"{rate[0]*100:.2f},{rate[1]*100:.2f}\n")

# Query 19

In [136]:
import pandas as pd
q19_unagg = pd.read_csv('/mydata/gt_query_19.csv')
q19_unagg

,sum_1,l_pageid,p_pageid
0,46491.4080,280,2911102
1,20941.4296,1296,1130761
2,7198.1469,1598,4085425
3,14817.8976,2458,56468
4,3213.5874,5289,1081953
...,...,...,...
113633,15709.4868,115509379,3486908
113634,26448.1972,115511029,1561327
113635,5381.4204,115513087,3317289
113636,40600.8720,115513441,2101589


In [139]:
y = compute_y(q19_unagg, ["l_pageid", "p_pageid"], [], ["sum_1"])

In [31]:
# lineitem: 1
# orders: 2
y1 = (q19_unagg.groupby(["l_pageid"]).agg({"sum_1": "sum"}) ** 2).agg({"sum_1": "sum"})
y2 = (q19_unagg.groupby(["p_pageid"]).agg({"sum_1": "sum"}) ** 2).agg({"sum_1": "sum"})
y12 = (q19_unagg.groupby(["l_pageid", "p_pageid"]).sum() ** 2).agg({"sum_1": "sum"})
y1, y2, y12

(sum_1    9.644635e+13
 dtype: float64,
 sum_1    3.705075e+14
 dtype: float64,
 sum_1    9.636764e+13
 dtype: float64)

In [140]:
from scipy.stats import norm
z_val = norm.ppf(0.975)
# var = lambda p1, p2: (p2 - p1*p2) / (p1*p2) * y1 + (p1 - p1*p2) / (p1*p2) * y2 + (1 - p1 - p2 + p1*p2) / (p1*p2) * y12
max_var = (0.05 * q19_unagg.agg({"sum_1": "sum"}) /  z_val) ** 2
print(max_var)

sum_1    5.810408e+15
dtype: float64


In [141]:
search_space = [0.0001 * i for i in range(1,10)] + [0.001 * i for i in range(1,10)] + [0.01 * i for i in range(1,10)] + [0.1 * i for i in range(1,10)]
# sample lineitem only
for p1 in search_space:
    current_var = var_func([p1, 1], y)
    if (current_var <= max_var).all().all():
        lineitem_only = (p1, 1)
        break
print(lineitem_only)

(0.02, 1)


In [142]:
# sample orders only
for p2 in search_space:
    current_var = var_func([1, p2], y)
    if (current_var <= max_var).all().all():
        part_only = (1, p2)
        break
print(part_only)

(1, 0.06)


In [143]:
lineitem_part = []
for p1 in search_space:
    for p2 in search_space:
        current_var = var_func([p1, p2], y)
        if (current_var <= max_var).all().all():
            lineitem_part.append((p1, p2))
            break
print(lineitem_part)

[(0.02, 0.9), (0.03, 0.6000000000000001), (0.04, 0.5), (0.05, 0.4), (0.06, 0.4), (0.07, 0.30000000000000004), (0.08, 0.30000000000000004), (0.09, 0.30000000000000004), (0.1, 0.30000000000000004), (0.2, 0.2), (0.30000000000000004, 0.1), (0.4, 0.09), (0.5, 0.08), (0.6000000000000001, 0.08), (0.7000000000000001, 0.07), (0.8, 0.07), (0.9, 0.07)]


In [144]:
possible_sample_rates = [lineitem_only, part_only] + lineitem_part
with open("q19_sample_rates.csv", "w") as f:
    f.write("p1,p2\n")
    for rate in possible_sample_rates:
        f.write(f"{rate[0]*100:.2f},{rate[1]*100:.2f}\n")

# Query 5

In [170]:
q5_unagg = pd.read_csv('/mydata/gt_query_5.csv')
q5_unagg

,n_name,sum_1,l_pageid,o_pageid,s_pageid,c_pageid
0,CHINA,13873.3668,12,3,214409,2588503
1,CHINA,72690.6180,121,27,186299,1703568
2,CHINA,45979.7580,151,35,220469,2953228
3,CHINA,89343.6110,224,52,165779,2841986
4,CHINA,13281.1560,234,54,219995,3565629
...,...,...,...,...,...,...
7282303,VIETNAM,50839.8800,115517549,26843200,212619,3303132
7282304,VIETNAM,16274.1600,115517571,26843205,185065,2169502
7282305,VIETNAM,11401.4673,115517641,26843221,15951,2327618
7282306,VIETNAM,42675.8150,115517652,26843224,48565,1101411


In [175]:
y = compute_y(q5_unagg, ["l_pageid", "o_pageid", "c_pageid", "s_pageid"], ["n_name"], ["sum_1"])

In [176]:
from scipy.stats import norm
z_val = norm.ppf(0.975)
max_var = (0.05 * q5_unagg.groupby("n_name").agg({"sum_1": "sum"}) /  z_val) ** 2
max_var

,sum_1
n_name,
CHINA,1.818614e+18
INDIA,1.810656e+18
INDONESIA,1.830311e+18
JAPAN,1.818328e+18
VIETNAM,1.826832e+18


In [177]:
import itertools
from tqdm import tqdm
search_space = [0.0001 * i for i in range(1,10)] + [0.001 * i for i in range(1,10)] + [0.01 * i for i in range(1,10)] + [0.1 * i for i in range(1,10)]
table_ids = [0, 1, 2, 3]
possible_sample_rates = []
# iterate over every possible subset of table ids
subsets = []
for i in range(1, 2**len(table_ids)):
    subset = [table_ids[j] for j in range(len(table_ids)) if i & (1 << j)]
    subsets.append(subset)
subsets = sorted(subsets, key=lambda x: len(x))
for subset in tqdm(subsets):
    if len(subset) == 1:
        for p in search_space:
            current_p = [p if i in subset else 1 for i in table_ids]
            current_var = var_func(current_p, y)
            if (current_var <= max_var).all().all():
                possible_sample_rates.append(current_p)
                break
    elif len(subset) == 2:
        if 0 not in subset:
            continue
        for p1 in search_space:
            for p2 in search_space:
                current_p = [1. for _ in table_ids]
                current_p[subset[0]] = p1
                current_p[subset[1]] = p2
                current_var = var_func(current_p, y)
                if (current_var <= max_var).all().all():
                    possible_sample_rates.append(current_p)
                    break
possible_sample_rates

100%|██████████| 15/15 [00:08<00:00,  1.69it/s]


[[0.002, 1, 1, 1],
 [1, 0.002, 1, 1],
 [1, 1, 0.003, 1],
 [1, 1, 1, 0.02],
 [0.002, 0.9, 1.0, 1.0],
 [0.003, 0.6000000000000001, 1.0, 1.0],
 [0.004, 0.5, 1.0, 1.0],
 [0.005, 0.4, 1.0, 1.0],
 [0.006, 0.30000000000000004, 1.0, 1.0],
 [0.007, 0.30000000000000004, 1.0, 1.0],
 [0.008, 0.30000000000000004, 1.0, 1.0],
 [0.009000000000000001, 0.2, 1.0, 1.0],
 [0.01, 0.2, 1.0, 1.0],
 [0.02, 0.09, 1.0, 1.0],
 [0.03, 0.06, 1.0, 1.0],
 [0.04, 0.05, 1.0, 1.0],
 [0.05, 0.04, 1.0, 1.0],
 [0.06, 0.03, 1.0, 1.0],
 [0.07, 0.03, 1.0, 1.0],
 [0.08, 0.03, 1.0, 1.0],
 [0.09, 0.02, 1.0, 1.0],
 [0.1, 0.02, 1.0, 1.0],
 [0.2, 0.009000000000000001, 1.0, 1.0],
 [0.30000000000000004, 0.006, 1.0, 1.0],
 [0.4, 0.005, 1.0, 1.0],
 [0.5, 0.004, 1.0, 1.0],
 [0.6000000000000001, 0.003, 1.0, 1.0],
 [0.7000000000000001, 0.003, 1.0, 1.0],
 [0.8, 0.003, 1.0, 1.0],
 [0.9, 0.002, 1.0, 1.0],
 [0.002, 1.0, 0.9, 1.0],
 [0.003, 1.0, 0.6000000000000001, 1.0],
 [0.004, 1.0, 0.5, 1.0],
 [0.005, 1.0, 0.4, 1.0],
 [0.006, 1.0, 0.3000000

In [178]:
with open("q5_sample_rates.csv", "w") as f:
    f.write("p1,p2,p3,p4\n")
    for rate in possible_sample_rates:
        f.write(",".join([f"{r*100:.2f}" for r in rate]) + "\n")

# Query 7

In [145]:
q7_unagg = pd.read_csv('/mydata/gt_query_7.csv')
q7_unagg

,supp_nation,cust_nation,l_year,sum_1,l_pageid,o_pageid,s_pageid,c_pageid
0,FRANCE,GERMANY,1995,75947.6008,169,39,37918,1316160
1,FRANCE,GERMANY,1995,76895.7750,172,40,175409,1082254
2,FRANCE,GERMANY,1995,30356.5104,237,55,51954,367515
3,FRANCE,GERMANY,1995,12100.9616,578,133,57213,2154840
4,FRANCE,GERMANY,1995,2214.4320,737,170,31799,2391971
...,...,...,...,...,...,...,...,...
5828728,GERMANY,FRANCE,1996,58790.3400,115517476,26843183,187868,2075882
5828729,GERMANY,FRANCE,1996,15136.8112,115517486,26843185,219527,2906597
5828730,GERMANY,FRANCE,1996,5755.8550,115517504,26843190,67200,2220144
5828731,GERMANY,FRANCE,1996,36843.2658,115517664,26843227,153580,1688979


In [49]:
page_cols = ["l_pageid", "o_pageid", "c_pageid", "s_pageid"]
group_cols = ["supp_nation", "cust_nation", "l_year"]
y1 = (q7_unagg.groupby(group_cols + [page_cols[0]]).agg({"sum_1": "sum"}) ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y12 = (q7_unagg.groupby(group_cols + page_cols[:2]).sum() ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y13 = (q7_unagg.groupby(group_cols + [page_cols[0], page_cols[2]]).agg({"sum_1": "sum"}) ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y14 = (q7_unagg.groupby(group_cols + [page_cols[0], page_cols[3]]).agg({"sum_1": "sum"}) ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y123 = (q7_unagg.groupby(group_cols + page_cols[:3]).sum() ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y124 = (q7_unagg.groupby(group_cols + [page_cols[0], page_cols[3]]).agg({"sum_1": "sum"}) ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y134 = (q7_unagg.groupby(group_cols + [page_cols[0], page_cols[2], page_cols[3]]).agg({"sum_1": "sum"}) ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y1234 = (q7_unagg.groupby(group_cols + page_cols).sum() ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y2 = (q7_unagg.groupby(group_cols + [page_cols[1]]).agg({"sum_1": "sum"}) ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y23 = (q7_unagg.groupby(group_cols + page_cols[1:3]).agg({"sum_1": "sum"}) ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y24 = (q7_unagg.groupby(group_cols + [page_cols[1], page_cols[3]]).agg({"sum_1": "sum"}) ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y234 = (q7_unagg.groupby(group_cols + page_cols[1:]).sum() ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y3 = (q7_unagg.groupby(group_cols + [page_cols[2]]).agg({"sum_1": "sum"}) ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y34 = (q7_unagg.groupby(group_cols + page_cols[2:]).sum() ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y4 = (q7_unagg.groupby(group_cols + [page_cols[3]]).sum() ** 2).groupby(group_cols).agg({"sum_1": "sum"})
y1, y12, y123, y1234, y2, y23, y24, y234, y3, y34, y4


(                                                                   sum_1
 supp_nation               cust_nation               l_year              
 FRANCE                    GERMANY                   1995    2.913552e+15
                                                     1996    2.924217e+15
 GERMANY                   FRANCE                    1995    2.927490e+15
                                                     1996    2.930422e+15,
                                                                    sum_1
 supp_nation               cust_nation               l_year              
 FRANCE                    GERMANY                   1995    2.911703e+15
                                                     1996    2.922255e+15
 GERMANY                   FRANCE                    1995    2.925769e+15
                                                     1996    2.928444e+15,
                                                                    sum_1
 supp_nation               cust_nati

In [146]:
y = compute_y(q7_unagg, page_cols, group_cols, ["sum_1"])
y

{(1,):                                                                    sum_1
 supp_nation               cust_nation               l_year              
 FRANCE                    GERMANY                   1995    2.913552e+15
                                                     1996    2.924217e+15
 GERMANY                   FRANCE                    1995    2.927490e+15
                                                     1996    2.930422e+15,
 (2,):                                                                    sum_1
 supp_nation               cust_nation               l_year              
 FRANCE                    GERMANY                   1995    3.004955e+15
                                                     1996    3.016175e+15
 GERMANY                   FRANCE                    1995    3.019879e+15
                                                     1996    3.022470e+15,
 (1,
  2):                                                                    sum_1
 supp_nation  

In [147]:
from scipy.stats import norm
z_val = norm.ppf(0.975)
max_var = (0.05 * q7_unagg.groupby(group_cols).agg({"sum_1": "sum"}) /  z_val) ** 2
max_var

sum_1
supp_nation               cust_nation               l_year              
FRANCE                    GERMANY                   1995    1.815287e+18
                                                    1996    1.823077e+18
GERMANY                   FRANCE                    1995    1.827814e+18
                                                    1996    1.830290e+18

In [168]:
import itertools
from tqdm import tqdm
search_space = [0.0001 * i for i in range(1,10)] + [0.001 * i for i in range(1,10)] + [0.01 * i for i in range(1,10)] + [0.1 * i for i in range(1,10)]
table_ids = [0, 1, 2, 3]
possible_sample_rates = []
# iterate over every possible subset of table ids
subsets = []
for i in range(1, 2**len(table_ids)):
    subset = [table_ids[j] for j in range(len(table_ids)) if i & (1 << j)]
    subsets.append(subset)
subsets = sorted(subsets, key=lambda x: len(x))
for subset in tqdm(subsets):
    if len(subset) == 1:
        for p in search_space:
            current_p = [p if i in subset else 1 for i in table_ids]
            current_var = var_func(current_p, y)
            if (current_var <= max_var).all().all():
                possible_sample_rates.append(current_p)
                break
    elif len(subset) == 2:
        if 0 not in subset:
            continue
        for p1 in search_space:
            for p2 in search_space:
                current_p = [1. for _ in table_ids]
                current_p[subset[0]] = p1
                current_p[subset[1]] = p2
                current_var = var_func(current_p, y)
                if (current_var <= max_var).all().all():
                    possible_sample_rates.append(current_p)
                    break
possible_sample_rates

100%|██████████| 15/15 [00:13<00:00,  1.09it/s]


[[0.002, 1, 1, 1],
 [1, 0.002, 1, 1],
 [1, 1, 0.003, 1],
 [1, 1, 1, 0.02],
 [0.002, 0.9, 1.0, 1.0],
 [0.003, 0.6000000000000001, 1.0, 1.0],
 [0.004, 0.5, 1.0, 1.0],
 [0.005, 0.4, 1.0, 1.0],
 [0.006, 0.30000000000000004, 1.0, 1.0],
 [0.007, 0.30000000000000004, 1.0, 1.0],
 [0.008, 0.30000000000000004, 1.0, 1.0],
 [0.009000000000000001, 0.2, 1.0, 1.0],
 [0.01, 0.2, 1.0, 1.0],
 [0.02, 0.09, 1.0, 1.0],
 [0.03, 0.06, 1.0, 1.0],
 [0.04, 0.05, 1.0, 1.0],
 [0.05, 0.04, 1.0, 1.0],
 [0.06, 0.03, 1.0, 1.0],
 [0.07, 0.03, 1.0, 1.0],
 [0.08, 0.03, 1.0, 1.0],
 [0.09, 0.02, 1.0, 1.0],
 [0.1, 0.02, 1.0, 1.0],
 [0.2, 0.009000000000000001, 1.0, 1.0],
 [0.30000000000000004, 0.006, 1.0, 1.0],
 [0.4, 0.005, 1.0, 1.0],
 [0.5, 0.004, 1.0, 1.0],
 [0.6000000000000001, 0.003, 1.0, 1.0],
 [0.7000000000000001, 0.003, 1.0, 1.0],
 [0.8, 0.003, 1.0, 1.0],
 [0.9, 0.002, 1.0, 1.0],
 [0.002, 1.0, 0.9, 1.0],
 [0.003, 1.0, 0.6000000000000001, 1.0],
 [0.004, 1.0, 0.4, 1.0],
 [0.005, 1.0, 0.4, 1.0],
 [0.006, 1.0, 0.3000000

In [169]:
with open("q7_sample_rates.csv", "w") as f:
    f.write("p1,p2,p3,p4\n")
    for rate in possible_sample_rates:
        f.write(",".join([f"{r*100:.2f}" for r in rate]) + "\n")

# Query 8

In [179]:
q8_unagg = pd.read_csv('/mydata/gt_query_8.csv')
q8_unagg

,o_year,sum_1,sum_2,l_pageid,p_pageid,o_pageid,c_pageid,s_pageid
0,1995,0.0000,25545.7143,15,1159629,3,3017547,202559
1,1995,0.0000,56576.9568,55,2842746,12,1868400,84500
2,1995,0.0000,45205.4460,165,2295261,38,3498279,212567
3,1995,0.0000,1241.2125,202,1864828,47,1382726,78951
4,1995,23262.8578,23262.8578,275,1996550,63,1604952,166245
...,...,...,...,...,...,...,...,...
2429192,1996,0.0000,16674.8820,115517469,542779,26843182,2329004,33376
2429193,1996,0.0000,27300.9600,115517544,1956339,26843199,1721293,122654
2429194,1996,0.0000,37107.1008,115517553,1184111,26843201,3224965,62560
2429195,1996,0.0000,37800.8400,115517607,1420048,26843213,2683789,151836


In [181]:
y = compute_y(q8_unagg, ["l_pageid", "o_pageid", "p_pageid", "c_pageid", "s_pageid"], ["o_year"], ["sum_1", "sum_2"])

In [185]:
from scipy.stats import norm
z_val = norm.ppf(0.975)
max_var = (0.05 * q8_unagg.groupby("o_year").agg({"sum_1": "sum", "sum_2": "sum"}) /  z_val) ** 2
max_var

,sum_1,sum_2
o_year,,
1995,2.031981e+15,1.263156e+18
1996,2.022147e+15,1.267919e+18


In [186]:
import itertools
from tqdm import tqdm
search_space = [0.0001 * i for i in range(1,10)] + [0.001 * i for i in range(1,10)] + [0.01 * i for i in range(1,10)] + [0.1 * i for i in range(1,10)]
table_ids = [0, 1, 2, 3]
possible_sample_rates = []
# iterate over every possible subset of table ids
subsets = []
for i in range(1, 2**len(table_ids)):
    subset = [table_ids[j] for j in range(len(table_ids)) if i & (1 << j)]
    subsets.append(subset)
subsets = sorted(subsets, key=lambda x: len(x))
for subset in tqdm(subsets):
    if len(subset) == 1:
        for p in search_space:
            current_p = [p if i in subset else 1 for i in table_ids]
            current_var = var_func(current_p, y)
            if (current_var <= max_var).all().all():
                possible_sample_rates.append(current_p)
                break
    elif len(subset) == 2:
        if 0 not in subset:
            continue
        for p1 in search_space:
            for p2 in search_space:
                current_p = [1. for _ in table_ids]
                current_p[subset[0]] = p1
                current_p[subset[1]] = p2
                current_var = var_func(current_p, y)
                if (current_var <= max_var).all().all():
                    possible_sample_rates.append(current_p)
                    break
possible_sample_rates

100%|██████████| 15/15 [00:11<00:00,  1.27it/s]


[[0.05, 1, 1, 1],
 [1, 0.05, 1, 1],
 [1, 1, 0.07, 1],
 [1, 1, 1, 0.05],
 [0.05, 0.9, 1.0, 1.0],
 [0.06, 0.7000000000000001, 1.0, 1.0],
 [0.07, 0.6000000000000001, 1.0, 1.0],
 [0.08, 0.6000000000000001, 1.0, 1.0],
 [0.09, 0.5, 1.0, 1.0],
 [0.1, 0.5, 1.0, 1.0],
 [0.2, 0.30000000000000004, 1.0, 1.0],
 [0.30000000000000004, 0.2, 1.0, 1.0],
 [0.4, 0.2, 1.0, 1.0],
 [0.5, 0.09, 1.0, 1.0],
 [0.6000000000000001, 0.07, 1.0, 1.0],
 [0.7000000000000001, 0.06, 1.0, 1.0],
 [0.8, 0.06, 1.0, 1.0],
 [0.9, 0.05, 1.0, 1.0],
 [0.05, 1.0, 0.9, 1.0],
 [0.06, 1.0, 0.8, 1.0],
 [0.07, 1.0, 0.7000000000000001, 1.0],
 [0.08, 1.0, 0.6000000000000001, 1.0],
 [0.09, 1.0, 0.5, 1.0],
 [0.1, 1.0, 0.5, 1.0],
 [0.2, 1.0, 0.30000000000000004, 1.0],
 [0.30000000000000004, 1.0, 0.2, 1.0],
 [0.4, 1.0, 0.2, 1.0],
 [0.5, 1.0, 0.2, 1.0],
 [0.6000000000000001, 1.0, 0.09, 1.0],
 [0.7000000000000001, 1.0, 0.08, 1.0],
 [0.8, 1.0, 0.08, 1.0],
 [0.9, 1.0, 0.07, 1.0],
 [0.05, 1.0, 1.0, 0.9],
 [0.06, 1.0, 1.0, 0.7000000000000001],
 [0

In [187]:
with open("q8_sample_rates.csv", "w") as f:
    f.write("p1,p2,p3,p4,p5\n")
    for rate in possible_sample_rates:
        f.write(",".join([f"{r*100:.2f}" for r in rate]) + "\n")

# Query 9

In [188]:
q9_unagg = pd.read_csv('/mydata/gt_query_9.csv')
q9_unagg